In this notebook, we will calculate features using Trotterization with IBM's QPU.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# This is necessary only when running the notebook in the docker container
%cd fourier_learning_ibm/

/root/workdir/fourier_learning_ibm


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
# GPU check
!nvidia-smi -L

GPU 0: NVIDIA A100-PCIE-40GB (UUID: GPU-3fac7503-bb55-35cd-f566-c919eaa55798)
GPU 1: NVIDIA A100-PCIE-40GB (UUID: GPU-bf03abcd-ef36-6733-da57-460c8eb9cab4)
GPU 2: NVIDIA A100-PCIE-40GB (UUID: GPU-bc01f8f3-0939-f2c0-6998-dd49d0360463)
GPU 3: NVIDIA A100-PCIE-40GB (UUID: GPU-b7aeac00-b332-1045-eebc-bb5653ac8402)


In [4]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import os
from heisenberg import (
    HeisenbergModel,
    get_graph,
    get_positions,
    # get_initial_layout,
    # get_prob0,
)
from setup import setup_backend
from helper_features import run_job, get_features
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Batch
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
import pickle
import networkx as nx
from datetime import datetime, timezone
import json
import math

In [5]:
import warnings

warnings.filterwarnings("ignore")

### Config backend

In [6]:
# density matrix simulator
backend_qpu, backend_dm_noiseless, backend_dm_noisy = setup_backend(
    qpu_name="ibm_marrakesh", device="GPU", method="density_matrix"
)

print()

# matrix product state simulator
backend_qpu, backend_mps_noiseless, backend_mps_noisy = setup_backend(
    qpu_name="ibm_marrakesh", device="GPU", method="matrix_product_state"
)

QiskitBackendNotFoundError: 'No backend matches the criteria.'

### Prepare parameters and load dataset

In [7]:
n_qubits = 12

C = 3
n_samples = 55
n_shots = 1000

if n_qubits <= 20:  # n_qubits <= 20
    threshold = 0.6

    # Load dataset (orient="records")
    if n_qubits == 20:  # 20 量子ビットの場合は、12 量子ビットの結果を使う
        df_n_step = pd.read_json(
            f"results/get_nstep/12Q/threshold{threshold:.2f}/optimal_n_step.json",
            orient="records",
        )
    else:
        df_n_step = pd.read_json(
            f"results/get_nstep/{n_qubits}Q/threshold{threshold:.2f}/optimal_n_step.json",
            orient="records",
        )
    display(df_n_step)

    # Note that K = n_features-1.
    # n_features counts only the number of real features, also the one of t=0, i.e., constant term.
    # The number of all features, including imaginary numbers, is 2K+1.
    n_features = len(df_n_step)

    times = df_n_step["t"].values
    n_step_array = df_n_step["n_step"].values

else:  # n_qubits > 20
    threshold = None
    n_features = 12
    # 間隔 np.pi/C で等間隔な点を n_features 個生成
    times = np.linspace(0, np.pi / C * (n_features - 1), n_features)

    # n_qubits > 20 の場合は、n_step は 1 から max_n_step の整数値を取る
    if n_qubits == 32:
        max_n_step = 4
    elif n_qubits == 40:
        max_n_step = 5
    # まず 1 から max_n_step までの n_features 個の等間隔の数値を生成
    n_step_array = np.linspace(1, max_n_step, n_features)
    # 各値を四捨五入して整数に変換
    n_step_array = [round(x) for x in n_step_array]

print(f"n_qubits: {n_qubits}")
print(f"C: {C}")
print(f"threshold: {threshold}")
print(f"n_samples: {n_samples}")
print(f"n_shots: {n_shots}")
print(f"n_features: {n_features}")
print(f"times: {times}")
print(f"n_step_array: {n_step_array}")

# Load dataset (orient="records")
df_dataset = pd.read_json(
    f"results/get_dataset/{n_qubits}Q/dataset.json",
    orient="records",
)
all_Js = df_dataset["Js"].values
all_expected_values = df_dataset["expected_value"].values
n_samples = len(all_Js)
display(df_dataset)

,t,n_step,fidelity
0,0.000000,1,1.000000
1,1.047198,1,0.876665
2,2.094395,1,0.874939
3,3.141593,1,0.865199
4,4.188790,1,0.830213
5,5.235988,2,0.770913
6,6.283185,2,0.762874
7,7.330383,2,0.744273
8,8.377580,2,0.706917
9,9.424778,3,0.670175


n_qubits: 12
C: 3
threshold: 0.6
n_samples: 55
n_shots: 1000
n_features: 12
times: [ 0.          1.04719755  2.0943951   3.14159265  4.1887902   5.23598776
  6.28318531  7.33038286  8.37758041  9.42477796 10.47197551 11.51917306]
n_step_array: [1 1 1 1 1 2 2 2 2 3 3 3]


,sample,Js,expected_value,expected_value_diag,diff
0,0,"[0.0954419343, -0.0212937804, 0.1249298174, 0....",0.792109,0.792109,9.820000e-08
1,1,"[0.1656462529, 0.0558403774, 0.1252779708, -0....",1.102887,1.102887,1.869000e-07
2,2,"[0.1404127689, 0.11727099390000001, 0.08304389...",1.130663,1.130663,1.148000e-07
3,3,"[-0.061944959, -0.014558126800000001, -0.14849...",1.552866,1.552865,3.841000e-07
4,4,"[0.11249198690000001, 0.1031829549, -0.0380960...",1.149036,1.149036,2.388000e-07
5,5,"[0.1515760814, -0.0221829062, 0.0371159373, -0...",1.332599,1.332599,2.610000e-08
6,6,"[0.022335651800000002, -0.07395899900000001, -...",1.114135,1.114134,4.651000e-07
7,7,"[0.0633589424, 0.022317037, 0.1110911404000000...",1.034106,1.034106,8.054000e-07
8,8,"[-0.1461354535, 0.00045074100000000005, -0.150...",1.421492,1.421492,1.777000e-07
9,9,"[0.1377260837, 0.1218283242, 0.0595475655, -0....",0.634733,0.634733,5.380000e-08


# Create Fourier feature

## Trotter simulation

In [8]:
config = {
    "n_qubits": n_qubits,
    "n_samples": n_samples,
    "n_shots": n_shots,
    "n_features": n_features,
    "times": times,
    "all_Js": all_Js,
    "n_step_array": n_step_array,
    "backend": backend_qpu,  # This is important!!
    "all_expected_values": all_expected_values,
}

In [9]:
# For QPU, we need batch instance and job_ids instead of job instances
batch, _, job_ids = run_job(config, backend_qpu, sim_type="qpu")

In [13]:
path = f"results/fourier_feature_qpu/{n_qubits}Q"

In [11]:
# Save job_ids
with open(f"{path}/job_ids.txt", "w") as f:
    for job_id in job_ids:
        f.write(f"{job_id}\n")

In [12]:
batch.status()

'In progress, not accepting new jobs'

In [15]:
features_df = get_features(config, jobs)
display(features_df)

Post-processing sample 0/55
Post-processing sample 1/55
Post-processing sample 2/55
Post-processing sample 3/55
Post-processing sample 4/55
Post-processing sample 5/55
Post-processing sample 6/55
Post-processing sample 7/55
Post-processing sample 8/55
Post-processing sample 9/55
Post-processing sample 10/55
Post-processing sample 11/55
Post-processing sample 12/55
Post-processing sample 13/55
Post-processing sample 14/55
Post-processing sample 15/55
Post-processing sample 16/55
Post-processing sample 17/55
Post-processing sample 18/55
Post-processing sample 19/55
Post-processing sample 20/55
Post-processing sample 21/55
Post-processing sample 22/55
Post-processing sample 23/55
Post-processing sample 24/55
Post-processing sample 25/55
Post-processing sample 26/55
Post-processing sample 27/55
Post-processing sample 28/55
Post-processing sample 29/55
Post-processing sample 30/55
Post-processing sample 31/55
Post-processing sample 32/55
Post-processing sample 33/55
Post-processing sample 3

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,...,f_7 Im,f_8 Re,f_8 Im,f_9 Re,f_9 Im,f_10 Re,f_10 Im,f_11 Re,f_11 Im,expected_value
0,0,0.551,0.030461,-0.119675,-0.001360,-0.058448,0.005464,-0.068660,0.010443,-0.052230,...,-0.007505,-0.016591,0.007054,-0.019712,-0.005236,-0.008287,0.005597,-0.006555,0.006785,0.913388
1,1,0.579,0.090748,0.024141,0.049545,0.024602,0.051107,0.034410,0.044573,0.050579,...,0.010586,0.002931,0.011332,0.004586,0.006631,0.007608,0.000344,-0.001471,0.002416,1.057062
2,2,0.579,0.125985,0.004457,0.047058,-0.004747,0.045491,0.002138,0.042955,0.009739,...,-0.000838,0.024162,-0.005405,0.005903,0.012576,0.001453,0.012645,0.009955,0.000944,1.009956
3,3,0.662,-0.000749,-0.109236,0.005293,-0.057324,0.012184,-0.086813,0.016147,-0.068617,...,-0.010753,-0.004051,-0.014986,0.000691,-0.019223,0.000254,-0.008540,-0.001694,-0.008314,1.115899
4,4,0.625,0.045894,-0.121782,-0.009468,-0.088059,-0.036764,-0.082537,-0.006860,-0.070837,...,-0.005206,-0.017433,-0.004011,-0.021145,-0.007867,-0.008997,0.008892,-0.010416,-0.000709,0.906205
5,5,0.658,0.143785,-0.023998,0.105586,0.017451,0.096718,0.003267,0.077586,0.021081,...,0.010731,0.000274,0.017690,0.021904,0.020596,0.006656,0.003702,0.002842,0.010046,0.956448
6,6,0.639,0.145525,0.011598,0.073844,0.033662,0.048060,0.025851,0.062572,0.041071,...,0.008485,-0.010836,0.010704,-0.006154,0.020276,-0.001408,0.007682,-0.010400,0.008357,1.192622
7,7,0.530,0.121075,-0.009050,0.075677,0.021190,0.091292,0.031157,0.080521,0.016535,...,0.008811,0.005863,0.002372,0.020524,0.007467,0.015929,0.014293,-0.004847,0.009618,0.961146
8,8,0.591,0.138799,0.011615,0.052548,-0.000829,0.067597,0.008408,0.060088,0.001551,...,0.004362,0.018077,-0.002493,-0.000330,0.004110,0.002687,0.004334,0.005172,0.002692,0.915409
9,9,0.607,0.055749,-0.115000,0.040097,-0.075982,0.037903,-0.080345,0.012268,-0.055808,...,-0.015499,-0.002867,-0.015026,-0.008546,-0.007547,0.001105,-0.007986,-0.002980,-0.006413,0.926034
